# GameStats2Text Main Notebook

This notebook handles data setup, model initialization, training, saving, and generation for the GameStats2Text generator model.

In [3]:
import os
import torch
from torch.utils.data import DataLoader, random_split
from src.process.setupData import GameStatsTextDataset, collate_fn
from src.models.generator import GameStats2TextGenerator
from transformers import GPT2Tokenizer

## Configuration

Set your paths, hyperparameters, and fusion method here.

In [4]:
DATA_CSV = 'data/dataset.csv'
CHECKPOINT_DIR = 'checkpoints'
BATCH_SIZE = 8
LR = 1e-4
EPOCHS = 3
FUSION_METHOD = 'concat'  # or 'add'

## Data Setup

Load the dataset, split into training and validation sets, and create DataLoaders.

In [5]:
dataset = GameStatsTextDataset(csv_file=DATA_CSV)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

/Users/jasonnair/Library/Caches/pypoetry/virtualenvs/game-stat-2-text-sARd-oMH-py3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Model Initialization

Instantiate the generator model and move it to the appropriate device.

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

stats_input_dim = len(dataset.feature_cols)
model = GameStats2TextGenerator(
    stats_input_dim=stats_input_dim,
    fusion_method=FUSION_METHOD
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GameStats2TextGenerator(
  (stats_encoder): StatsEncoder(
    (encoder): Sequential(
      (0): Linear(in_features=10, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=128, out_features=64, bias=True)
      (4): ReLU(inplace=True)
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=64, out_features=32, bias=True)
    )
  )
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D(nf=2304, nx=768)
            (c_proj): Conv1D(nf=768, nx=768)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNo

## Training Loop

Run training and validation for a set number of epochs.

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    # Training
    model.train()
    total_train_loss = 0.0
    for batch in train_loader:
        stats = batch['stats'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(stats, input_ids, attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            stats = batch['stats'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(stats, input_ids, attention_mask, labels=labels)
            total_val_loss += outputs.loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch} | Val   Loss: {avg_val_loss:.4f}")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


## Save Checkpoint

Save the trained model for later use.

In [ ]:
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, 'generator.pt'))
print('Model saved to', os.path.join(CHECKPOINT_DIR, 'generator.pt'))

## Generation

Load the checkpoint and generate a sample response given a prompt.

In [ ]:
from src.process.setupData import GameStatsTextDataset
from src.models.generator import GameStats2TextGenerator
from transformers import GPT2Tokenizer
import os, torch

dataset = GameStatsTextDataset(csv_file=DATA_CSV)
sample_stats = dataset.stats[-1]

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GameStats2TextGenerator(
    stats_input_dim=len(dataset.feature_cols),
    fusion_method=FUSION_METHOD
)
model.load_state_dict(
    torch.load(os.path.join(CHECKPOINT_DIR, 'generator.pt'), map_location=device)
)
model.to(device).eval()

prompt = "How do you feel about your performance tonight?"
response = model.generate(
        stats=sample_stats,
        prompt=prompt,
        tokenizer=tokenizer)
print("\n--- GENERATED RESPONSE ---\n", response)
